# Homework 6: Data Preprocessing
Apply cleaning functions and save processed data.

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Add parent directory to path to import src module
sys.path.append('..')

from src import cleaning


## Load Raw Dataset

In [ ]:
# Load raw dataset
df = pd.read_csv('../data/raw/api_source-alpha_symbol-AAPL_20250817-230500.csv')
print(f"Loaded dataset with shape: {df.shape}")
df.head()

Loaded dataset with shape: (100, 2)


,date,adj_close
0,2025-03-25,223.75
1,2025-03-26,221.53
2,2025-03-27,223.85
3,2025-03-28,217.90
4,2025-03-31,222.13


## Simulate Missingness (MCAR/MAR/MNAR)


In [ ]:
# Rename column for clarity
df = df.rename(columns={'adj_close': 'price'})

# Add some additional columns for demonstration
np.random.seed(42)
df['volume'] = np.random.randint(1000000, 50000000, len(df))
df['market_cap'] = df['price'] * 15800000000  # Approximate market cap

# Simulate different types of missingness following lecture pattern

# MCAR: random missing (10% of volume data)
df['volume_MCAR'] = df['volume'].mask(np.random.rand(len(df)) < 0.1)

# MAR: missing depends on another column (missing when price > 230)
df['market_cap_MAR'] = df['market_cap'].mask(df['price'] > 230)

# MNAR: missing depends on its own value (high volumes are missing)
df['volume_MNAR'] = df['volume'].mask(df['volume'] > 40000000)

print("Simulated missingness:")
print("MCAR (volume):", df['volume_MCAR'].isna().sum())
print("MAR (market_cap):", df['market_cap_MAR'].isna().sum()) 
print("MNAR (volume):", df['volume_MNAR'].isna().sum())

df.head()


Simulated missingness:
MCAR (volume): 11
MAR (market_cap): 3
MNAR (volume): 21


,date,price,volume,market_cap,volume_MCAR,market_cap_MAR,volume_MNAR
0,2025-03-25,223.75,22081788,3.535250e+12,22081788.0,3.535250e+12,22081788.0
1,2025-03-26,221.53,14315092,3.500174e+12,14315092.0,3.500174e+12,14315092.0
2,2025-03-27,223.85,36788921,3.536830e+12,36788921.0,3.536830e+12,36788921.0
3,2025-03-28,217.90,27735830,3.442820e+12,NaN,3.442820e+12,27735830.0
4,2025-03-31,222.13,49140618,3.509654e+12,49140618.0,3.509654e+12,NaN


## Apply Cleaning Functions

In [ ]:
# Apply cleaning functions step by step

# Step 1: Fill missing values with median for numerical columns
print("1. Filling missing values with median...")
numeric_cols = ['volume_MCAR', 'market_cap_MAR', 'volume_MNAR']
df_cleaned = cleaning.fill_missing_median(df, numeric_cols)

# Step 2: Drop rows with excessive missing values (threshold=0.5 means keep rows with 50%+ non-null)
print("\n2. Dropping rows with excessive missing values...")
df_cleaned = cleaning.drop_missing(df_cleaned, threshold=0.5)

# Step 3: Normalize numerical data to 0-1 range
print("\n3. Normalizing data...")
cols_to_normalize = ['price', 'volume', 'market_cap']
df_cleaned = cleaning.normalize_data(df_cleaned, cols_to_normalize)

print(f"\nCleaning complete! Shape: {df_cleaned.shape}")
df_cleaned.head()

1. Filling missing values with median...
Filled 11 missing values in 'volume_MCAR' with median: 23857742.00
Filled 3 missing values in 'market_cap_MAR' with median: 3241686000000.00
Filled 21 missing values in 'volume_MNAR' with median: 21546531.00

2. Dropping rows with excessive missing values...
Dropped 0 rows with <50.0% non-null values

3. Normalizing data...
Normalized 'price' to range [0, 1]
Normalized 'volume' to range [0, 1]
Normalized 'market_cap' to range [0, 1]

Cleaning complete! Shape: (100, 7)


,date,price,volume,market_cap,volume_MCAR,market_cap_MAR,volume_MNAR
0,2025-03-25,0.842719,0.430432,0.842719,22081788.0,3.535250e+12,22081788.0
1,2025-03-26,0.806272,0.271488,0.806272,14315092.0,3.500174e+12,14315092.0
2,2025-03-27,0.844361,0.731411,0.844361,36788921.0,3.536830e+12,36788921.0
3,2025-03-28,0.746675,0.546141,0.746675,23857742.0,3.442820e+12,27735830.0
4,2025-03-31,0.816122,0.984186,0.816122,49140618.0,3.509654e+12,21546531.0


## Compare Original vs Cleaned Data


In [ ]:
# Compare datasets
print("COMPARISON: Original vs Cleaned")
print("=" * 40)
print(f"Shape: {df.shape} -> {df_cleaned.shape}")
print(f"Missing values: {df.isna().sum().sum()} -> {df_cleaned.isna().sum().sum()}")

print("\nMissing values by column:")
print("Original:")
print(df.isna().sum()[df.isna().sum() > 0])
print("\nCleaned:")  
print(df_cleaned.isna().sum()[df_cleaned.isna().sum() > 0])

print("\nSample statistics (before/after):")
for col in ['price', 'volume', 'market_cap']:
    if col in df.columns and col in df_cleaned.columns:
        print(f"{col}: {df[col].mean():.2f} -> {df_cleaned[col].mean():.2f}")


COMPARISON: Original vs Cleaned
Shape: (100, 7) -> (100, 7)
Missing values: 35 -> 0

Missing values by column:
Original:
volume_MCAR       11
market_cap_MAR     3
volume_MNAR       21
dtype: int64

Cleaned:
Series([], dtype: int64)

Sample statistics (before/after):
price: 207.04 -> 0.57
volume: 25746426.66 -> 0.51
market_cap: 3271238320000.00 -> 0.57


## Save Cleaned Dataset

In [ ]:
# Save cleaned dataset
df_cleaned.to_csv('../data/processed/sample_data_cleaned.csv', index=False)
print("Saved cleaned dataset to ../data/processed/sample_data_cleaned.csv")

## Preprocessing Assumptions

**Missing Data Handling:**
- Filled missing values with median assuming MCAR/MAR patterns
- Median is robust to outliers compared to mean
- Assumes missing values don't carry information

**Row Dropping:**
- Dropped rows with <50% non-null values
- Assumes these rows have insufficient information
- May lose valid but sparse data points

**Normalization:**
- Used MinMax scaling to [0,1] range
- Assumes min/max values are representative, not outliers
- Makes features comparable in scale for analysis

**Data Quality:**
- Original dataset had stock price data from Alpha Vantage
- Added simulated volume and market cap for demonstration
- Simulated different missingness patterns for learning purposes
